In [ ]:
import pandas as pd
import numpy as np
import sqlalchemy
from sqlqueries_unc_preeclampsia import *
import os
import dotenv

### 1. load bp data

In [ ]:
dotenv.load_dotenv()
connection_string = f"mssql+pymssql://@{os.getenv('HOSTNAME')}/{os.getenv('DATABASE')}"
engine = sqlalchemy.create_engine(connection_string)
sql_string = f'''
select
    *
from {os.getenv('DATABASE')}.{os.getenv('SCHEMA')}.rpt_preeclampsia__bp_time_series
'''
dat = pd.read_sql(sql_string, con=engine)

In [ ]:
dat_with_chtn_any = dat[dat['chtn_any']==1].groupby(['cp_anypree_sf', 'preg_weeks'])[['systolic', 'diastolic']].agg(['mean', 'std']).reset_index()
dat_without_chtn_any = dat[dat['chtn_any']==0].groupby(['cp_anypree_sf', 'preg_weeks'])[['systolic', 'diastolic']].agg(['mean', 'std']).reset_index()
dat_with_chtn_any

In [ ]:
from matplotlib import pyplot as plt

idx = (dat_with_chtn_any['cp_anypree_sf'] == 1)
fig, axes = plt.subplots(2,1)
axes = axes.flatten()
axes[0].plot(dat_with_chtn_any.loc[idx, 'preg_weeks'], dat_with_chtn_any.loc[idx, ('systolic', 'mean')], 'r-', label='cp_anypree_sf=1')
axes[0].plot(dat_with_chtn_any.loc[~idx, 'preg_weeks'], dat_with_chtn_any.loc[~idx, ('systolic', 'mean')], 'k-', label='cp_anypree_sf=0')
axes[1].plot(dat_with_chtn_any.loc[idx, 'preg_weeks'], dat_with_chtn_any.loc[idx, ('diastolic', 'mean')], 'r-')
axes[1].plot(dat_with_chtn_any.loc[~idx, 'preg_weeks'], dat_with_chtn_any.loc[~idx, ('diastolic', 'mean')], 'k-')
axes[0].legend()
plt.suptitle('chtn_any=1')

In [ ]:
idx = (dat_without_chtn_any['cp_anypree_sf'] == 1)
fig, axes = plt.subplots(2,1)
axes = axes.flatten()
axes[0].plot(dat_without_chtn_any.loc[idx, 'preg_weeks'], dat_without_chtn_any.loc[idx, ('systolic', 'mean')], 'r-', label='cp_anypree_sf=1')
axes[0].plot(dat_without_chtn_any.loc[~idx, 'preg_weeks'], dat_without_chtn_any.loc[~idx, ('systolic', 'mean')], 'k-', label='cp_anypree_sf=0')
axes[1].plot(dat_without_chtn_any.loc[idx, 'preg_weeks'], dat_without_chtn_any.loc[idx, ('diastolic', 'mean')], 'r-')
axes[1].plot(dat_without_chtn_any.loc[~idx, 'preg_weeks'], dat_without_chtn_any.loc[~idx, ('diastolic', 'mean')], 'k-')
axes[0].legend()
plt.suptitle('chtn_any=0')

In [ ]:
from matplotlib import pyplot as plt

idx = (dat_with_chtn_any['cp_anypree_sf'] == 1)
plt.errorbar(dat_with_chtn_any.loc[idx, 'preg_weeks'], 
                dat_with_chtn_any.loc[idx, ('systolic', 'mean')],
                dat_with_chtn_any.loc[idx, ('systolic', 'std')],
                fmt='r-', label='cp_anypree_sf=1')
plt.errorbar(dat_with_chtn_any.loc[~idx, 'preg_weeks'], 
                dat_with_chtn_any.loc[~idx, ('systolic', 'mean')],
                dat_with_chtn_any.loc[~idx, ('systolic', 'std')],
                fmt='k-', label='cp_anypree_sf=0')
plt.legend()
plt.title('chtn_any=1')